# Hierarchical Naive Bayes Model

Load the labeled dataset.

In [ ]:
import pandas as pd

In [ ]:
# Load labeled dataset
df = pd.read_csv('data/labeledident.csv')

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

In [ ]:
# Create the Hierarchical Naive Bayes model
model = BayesianModel()

In [ ]:
level1 = ['Identity Match']
level2 = ['Name Match', 'DOB Match', 'Address Match', 'ID Match']
level3 = ['First Name Match', 'Middle Name Match', 'Last Name Match', 'DOB Similarity', 'Address Similarity', 'ID Similarity']
level4 = ['First Name Similarity', 'Middle Name Similarity', 'Last Name Similarity']

# Add Nodes
model.add_nodes_from(level1)
model.add_nodes_from(level2)
model.add_nodes_from(level3)
model.add_nodes_from(level4)

# Add edges in most confusing way possible

# Connect level1 with level2
pairs = []
for val in level2:
    pairs.append(level1[0], val)
model.add_edges_from(pairs)

# Connect Name Match
pairs.clear()
for val in level3[0:3]:
    pairs.append(level2[0], val)
model.add_edges_from(pairs)

# Connect DOB, Address, and ID Match with Similarity
pairs.clear()
for i in range(1, 4):
    pairs.append(level2[i], level3[i + 2])
model.add_edges_from(pairs)

# Connect Name Matches with Similarity
pairs.clear()
for i in range(3):
    pairs.append(level3[i], level4[i])
model.add_edges_from(pairs)

In [ ]:
# Estimate CPDs using Maximum Likelihood Estimation (MLE)
estimator = MaximumLikelihoodEstimator(model, data)

In [ ]:
# Fit the model to the data
model.fit(data, estimator=estimator)

In [ ]:
# Predict using the trained model
# Assuming you have a test DataFrame 'test_data' with columns 'A', 'B', 'C', 'D', 'E'
predictions = model.predict(test_data)